In [ ]:
def find_json(img_folder, img):
    image_path = os.path.join(img_folder, img)
    image_name, image_ext = os.path.splitext(img)
    original_json = image_name + '.json'
    json_path = os.path.join(img_folder, 'json', original_json)
    # print(json_path)
    img = cv2.imread(image_path)
    # os.remove(image_path)
    # cv2.imshow("result", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img, json_path, image_path

In [ ]:
def check_bbox_scale(json_path, max_p):
    with open(json_path, 'r') as f:
        data = json.load(f)
    bboxes = data.get('bboxes', [])
    bbox = bboxes[0]
    _, _, w, h = bbox[:4]
    face_size = (w * h) / (224 * 224)
    if face_size * pow(max_p, 2) < 0.02469:
        return face_size, False
    else: 
        return face_size, True

In [ ]:
def modify_paste_json(paste_json_path, resize, location_x, location_y):
    with open(paste_json_path, 'r') as f:
        data = json.load(f)
    bboxes = data.get('bboxes', [])
    bbox = bboxes[0]
    x, y, w, h = bbox[:4]
    x, y, w, h = (x - 40) * resize, y * resize, w * resize, h * resize
    x = location_x + x
    y = location_y + y
    box = [int(x), int(y), int(w), int(h), 1, bbox[-1]]
    return box

### Augmentation

In [ ]:
def simulate_backlighting(image, intensity=0.3):
    # 轉換影像至 HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
   
    # 分離 HSV 頻道
    h, s, v = cv2.split(hsv)
   
    # 設定光源位置 (背光，從上到下)
    rows, cols = image.shape[:2]
    mask = np.zeros((rows, cols), np.uint8)
    cv2.ellipse(mask, (cols // 2, rows // 4), (cols // 2, rows // 2), 0, 0, 0, (0, 0, 255), 2)
    # 遮罩區域增加亮度，其他區域減少亮度
    v = cv2.add(v, (mask * intensity).astype(np.uint8))
    v = cv2.subtract(v, ((1 - mask / 255) * (255 * intensity)).astype(np.uint8))
   
    # 合併 HSV 頻道並轉換回 BGR
    backlit_hsv = cv2.merge([h, s, v])
    backlit_image = cv2.cvtColor(backlit_hsv, cv2.COLOR_HSV2BGR)
   
    return backlit_image

In [ ]:
def add_random_lens_flare(image):
    h, w = image.shape[:2]
    # Generate a random position for the flare
    center_x = random.randint(int(0.3 * w), int(0.7 * w))
    center_y = random.randint(int(0.3 * h), int(0.7 * h))
   
    # Create an overlay to apply the flare
    overlay = image.copy()
   
    # Apply a bright circle to simulate the flare
    cv2.circle(overlay, (center_x, center_y), random.randint(50, 150), (255, 255, 255), -1)
   
    # Apply Gaussian blur to make it more realistic
    overlay = cv2.GaussianBlur(overlay, (0, 0), random.uniform(15, 50))
   
    # Combine with the original image
    alpha = random.uniform(0.1, 0.3)
    image_with_flare = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)
   
    return image_with_flare


### Paste one people in one image

In [ ]:
import cv2
import random
import os
import json
import numpy as np

# 讀取 448x448 的方形影像
def syn_1(image_dir, output_path):
    max_p = 0.9
    flag = False
    while flag == False:
        img = random.choice(os.listdir(image_dir))
        if not img.endswith('.jpg'):
            continue
        image, json_path, image_path = find_json(image_dir, img)
        min_p, flag = check_bbox_scale(json_path, max_p)
    image = cv2.resize(image[:, 40:-40], (224, 224))
    image_height, image_width = image.shape[:2]

    bg_dir = r'video\bg'
    bg_list = os.listdir(bg_dir)
    bg = os.path.join(bg_dir, random.choice(bg_list))
    background = cv2.imread(bg)
    # print(background.shape)
    if background.shape[0] != 288 and background.shape[1] != 208:
        background = cv2.resize(background[:, 212:-212], (288, 208))

    min_p = int(pow(0.02469 / min_p, 0.5) * 100)
    max_p *= 100
    resize_scale = random.randint(min_p, max_p) / 100
    # print(resize_scale)
    new_width = int(image_width * resize_scale)
    new_height = int(image_height * resize_scale)
    resized_image = cv2.resize(image, (new_width, new_height))

    max_x_offset = 288 - new_width
    max_y_offset = 208 - new_height
    x_offset = random.randint(0, max_x_offset)
    y_offset = random.randint(0, max_y_offset)

    background[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image
    new_bbox = modify_paste_json(json_path, resize_scale, x_offset, y_offset)

    # cv2.rectangle(background, (new_bbox[0], new_bbox[1]), (new_bbox[0] + new_bbox[2], new_bbox[1] + new_bbox[3]), (0, 255, 0), 2)
    # cv2.imshow('Result', background)
    # cv2.waitKey(0)
    # cv2.destroyAllWdows()

    output_json = os.path.join(output_path, 'json', img.split('.')[0] + '.json')
    # print("output_json: ", output_json)
    output_img = os.path.join(output_path, img)
    # print("output_img: ", output_img)
    data = {"bboxes": new_bbox}
    with open(output_json, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    if(random.random()>0.9):
        background = add_random_lens_flare(background)
    if(random.random()>0.9):
        background = simulate_backlighting(background)
    cv2.imwrite(output_img, background)


### Paste two people in one image

In [ ]:
import cv2
import random

def syn_2(image_dir1, image_dir2, output_path):
    max_p = 0.64
    flag = False
    while flag == False:
        img1 = random.choice(os.listdir(image_dir1))
        if not img1.endswith('.jpg'):
            continue
        image1, json_path1, image_path1 = find_json(image_dir1, img1)
        min_p1, flag = check_bbox_scale(json_path1, max_p)
    
    flag = False
    while flag == False:
        img2 = random.choice(os.listdir(image_dir2))
        if not img2.endswith('.jpg'):
            continue
        image2, json_path2, image_path2 = find_json(image_dir2, img2)
        min_p2, flag = check_bbox_scale(json_path2, max_p)
    
    image1 = cv2.resize(image1[:, 40:-40], (224, 224))
    image2 = cv2.resize(image2[:, 40:-40], (224, 224))
    image_height1, image_width1 = image1.shape[:2]
    image_height2, image_width2 = image2.shape[:2]

    bg_dir = r'video\bg'
    bg_list = os.listdir(bg_dir)
    bg = os.path.join(bg_dir, random.choice(bg_list))
    background = cv2.imread(bg)
    background = cv2.resize(background[:, 212:-212], (288, 208))

    # 影像1
    min_p1 = int(pow(0.02469 / min_p1, 0.5) * 100)
    max_p *= 100
    resize_scale1 = random.randint(min_p1, max_p) / 100
    # print(resize_scale1)
    new_width1 = int(image_width1 * resize_scale1)
    new_height1 = int(image_height1 * resize_scale1)
    resized_image1 = cv2.resize(image1, (new_width1, new_height1))
    max_x_offset1 = 144 - new_width1
    max_y_offset1 = 208 - new_height1
    x_offset1 = random.randint(0, max_x_offset1)
    y_offset1 = random.randint(0, max_y_offset1)
    # 影像2
    min_p2 = int(pow(0.02469 / min_p2, 0.5) * 100)
    # print(min_p2, max_p)
    resize_scale2 = random.randint(min_p2, max_p) / 100
    # print(resize_scale2)
    new_width2 = int(image_width2 * resize_scale2)
    new_height2 = int(image_height2 * resize_scale2)
    resized_image2 = cv2.resize(image2, (new_width2, new_height2))
    max_x_offset2 = 288 - new_width2
    max_y_offset2 = 208 - new_height2
    x_offset2 = random.randint(144, max_x_offset2)
    y_offset2 = random.randint(0, max_y_offset2)

    if(random.random()>0.9):
        resized_image1 = add_random_lens_flare(resized_image1)
    if(random.random()>0.9):
        resized_image1 = simulate_backlighting(resized_image1)
    if(random.random()>0.9):
        resized_image2 = add_random_lens_flare(resized_image2)
    if(random.random()>0.9):
        resized_image2 = simulate_backlighting(resized_image2)

    # 將縮放後的影像貼在背景上
    background[y_offset1:y_offset1 + new_height1, x_offset1:x_offset1 + new_width1] = resized_image1
    background[y_offset2:y_offset2 + new_height2, x_offset2:x_offset2 + new_width2] = resized_image2
    new_bbox1 = modify_paste_json(json_path1, resize_scale1, x_offset1, y_offset1)
    new_bbox2 = modify_paste_json(json_path2, resize_scale2, x_offset2, y_offset2)

    # 顯示結果影像和新的 bounding box
    # cv2.rectangle(background, (new_bbox1[0], new_bbox1[1]), (new_bbox1[0] + new_bbox1[2], new_bbox1[1] + new_bbox1[3]), (0, 255, 0), 2)
    # cv2.rectangle(background, (new_bbox2[0], new_bbox2[1]), (new_bbox2[0] + new_bbox2[2], new_bbox2[1] + new_bbox2[3]), (0, 255, 0), 2)

    # cv2.imshow('Result', background)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # 保存結果影像
    output_json = os.path.join(output_path, 'json', img1.split('.')[0] + '_' + img2.split('.')[0] + '.json')
    # print("output_json: ", output_json)
    output_img = os.path.join(output_path, img1.split('.')[0] + '_' + img2)
    # print("output_img: ", output_img)
    bbox = [new_bbox1, new_bbox2]
    data = {"bboxes": bbox}
    with open(output_json, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    # 保存結果影像
    cv2.imwrite(output_img, background)



### Paste three people in one image

In [ ]:
import cv2
import random

# 讀取 448x448 的方形影像
def syn_3(image_dir1, image_dir2, image_dir3, output_path):
    
    max_p = 0.5
    flag = False
    while flag == False:
        img1 = random.choice(os.listdir(image_dir1))
        if not img1.endswith('.jpg'):
            continue
        image1, json_path1, _ = find_json(image_dir1, img1)
        min_p1, flag = check_bbox_scale(json_path1, max_p)

    flag = False
    while flag == False:
        img2 = random.choice(os.listdir(image_dir2))
        if not img2.endswith('.jpg'):
            continue
        image2, json_path2, _ = find_json(image_dir2, img2)
        min_p2, flag = check_bbox_scale(json_path2, max_p)

    flag = False
    while flag == False:
        img3 = random.choice(os.listdir(image_dir3))
        if not img3.endswith('.jpg'):
            continue
        image3, json_path3, _ = find_json(image_dir3, img3)
        min_p3, flag = check_bbox_scale(json_path3, 0.47)

    image1 = cv2.resize(image1[:, 40:-40], (224, 224))
    image2 = cv2.resize(image2[:, 40:-40], (224, 224))
    image3 = cv2.resize(image3[:, 40:-40], (224, 224))
    image_height1, image_width1 = image1.shape[:2]
    image_height2, image_width2 = image2.shape[:2]
    image_height3, image_width3 = image3.shape[:2]

    bg_dir = r'video\bg'
    bg_list = os.listdir(bg_dir)
    bg = os.path.join(bg_dir, random.choice(bg_list))
    background = cv2.imread(bg)
    background = cv2.resize(background[:, 212:-212], (288, 208))

    # 影像1
    min_p1 = int(pow(0.02469 / min_p1, 0.5) * 100)
    max_p *= 100
    resize_scale1 = random.randint(min_p1, max_p) / 100
    # print(resize_scale1)
    new_width1 = int(image_width1 * resize_scale1)
    new_height1 = int(image_height1 * resize_scale1)
    resized_image1 = cv2.resize(image1, (new_width1, new_height1))
    max_x_offset1 = 192 - new_width1
    max_y_offset1 = 114 - new_height1
    x_offset1 = random.randint(0, max_x_offset1+10)
    y_offset1 = random.randint(0, max_y_offset1+10)
    # 影像2
    min_p2 = int(pow(0.02469 / min_p2, 0.5) * 100)
    resize_scale2 = random.randint(min_p2, max_p) / 100
    # print(resize_scale2)
    new_width2 = int(image_width2 * resize_scale2)
    new_height2 = int(image_height2 * resize_scale2)
    resized_image2 = cv2.resize(image2, (new_width2, new_height2))
    max_x_offset2 = 192 - new_width2
    max_y_offset2 = 208 - new_height2
    x_offset2 = random.randint(0, max_x_offset2+10)
    y_offset2 = random.randint(94, max_y_offset2)
    # 影像3
    min_p3 = int(pow(0.02469 / min_p3, 0.5) * 100)
    resize_scale3 = random.randint(min_p3, 47) / 100
    # print(resize_scale3)
    new_width3 = int(image_width3 * resize_scale3)
    new_height3 = int(image_height3 * resize_scale3)
    resized_image3 = cv2.resize(image3, (new_width3, new_height3))
    max_x_offset3 = 288 - new_width3
    max_y_offset3 = 208 - new_height3
    x_offset3 = random.randint(182, max_x_offset3)
    y_offset3 = random.randint(0, max_y_offset3)

    if(random.random()>0.9):
        resized_image1 = add_random_lens_flare(resized_image1)
    if(random.random()>0.9):
        resized_image1 = simulate_backlighting(resized_image1)
    if(random.random()>0.9):
        resized_image2 = add_random_lens_flare(resized_image2)
    if(random.random()>0.9):
        resized_image2 = simulate_backlighting(resized_image2)
    if(random.random()>0.9):
        resized_image3 = add_random_lens_flare(resized_image3)
    if(random.random()>0.9):
        resized_image3 = simulate_backlighting(resized_image3)


    # 將縮放後的影像貼在背景上
    background[y_offset1:y_offset1 + new_height1, x_offset1:x_offset1 + new_width1] = resized_image1
    background[y_offset2:y_offset2 + new_height2, x_offset2:x_offset2 + new_width2] = resized_image2
    background[y_offset3:y_offset3 + new_height3, x_offset3:x_offset3 + new_width3] = resized_image3
    new_bbox1 = modify_paste_json(json_path1, resize_scale1, x_offset1, y_offset1)
    new_bbox2 = modify_paste_json(json_path2, resize_scale2, x_offset2, y_offset2)
    new_bbox3 = modify_paste_json(json_path3, resize_scale3, x_offset3, y_offset3)

    # 顯示結果影像和新的 bounding box
    # cv2.rectangle(background, (new_bbox1[0], new_bbox1[1]), (new_bbox1[0] + new_bbox1[2], new_bbox1[1] + new_bbox1[3]), (0, 255, 0), 2)
    # cv2.rectangle(background, (new_bbox2[0], new_bbox2[1]), (new_bbox2[0] + new_bbox2[2], new_bbox2[1] + new_bbox2[3]), (0, 255, 0), 2)
    # cv2.rectangle(background, (new_bbox3[0], new_bbox3[1]), (new_bbox3[0] + new_bbox3[2], new_bbox3[1] + new_bbox3[3]), (0, 255, 0), 2)

    # cv2.imshow('Result', background)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    output_json = os.path.join(output_path, 'json', img1.split('.')[0] + '_' + img2.split('.')[0] + '_' + img3.split('.')[0] + '.json')
    # print("output_json: ", output_json)
    output_img = os.path.join(output_path, img1.split('.')[0] + '_' + img2.split('.')[0] + '_' + img3)
    # print("output_img: ", output_img)
    bbox = [new_bbox1, new_bbox2, new_bbox3]
    data = {"bboxes": bbox}
    with open(output_json, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    # 保存結果影像
    cv2.imwrite(output_img, background)


### Paste four people in one image

In [ ]:
import cv2
import random


def syn_4(image_dir1, image_dir2, image_dir3, image_dir4, output_path):
    max_p = 0.5
    flag = False
    while flag == False:
        img1 = random.choice(os.listdir(image_dir1))
        if not img1.endswith('.jpg'):
            continue
        image1, json_path1, _ = find_json(image_dir1, img1)
        min_p1, flag = check_bbox_scale(json_path1, max_p)

    flag = False
    while flag == False:
        img2 = random.choice(os.listdir(image_dir2))
        if not img2.endswith('.jpg'):
            continue
        image2, json_path2, _ = find_json(image_dir2, img2)
        min_p2, flag = check_bbox_scale(json_path2, max_p)

    flag = False
    while flag == False:
        img3 = random.choice(os.listdir(image_dir3))
        if not img3.endswith('.jpg'):
            continue
        image3, json_path3, _ = find_json(image_dir3, img3)
        min_p3, flag = check_bbox_scale(json_path3, max_p)

    
    flag = False
    while flag == False:
        img4 = random.choice(os.listdir(image_dir4))
        if not img4.endswith('.jpg'):
            continue
        image4, json_path4, _ = find_json(image_dir4, img4)
        min_p4, flag = check_bbox_scale(json_path4, max_p)

    image1 = cv2.resize(image1[:, 40:-40], (224, 224))
    image2 = cv2.resize(image2[:, 40:-40], (224, 224))
    image3 = cv2.resize(image3[:, 40:-40], (224, 224))
    image4 = cv2.resize(image4[:, 40:-40], (224, 224))
    image_height1, image_width1 = image1.shape[:2]
    image_height2, image_width2 = image2.shape[:2]
    image_height3, image_width3 = image3.shape[:2]
    image_height4, image_width4 = image4.shape[:2]

    bg_dir = r'video\bg'
    bg_list = os.listdir(bg_dir)
    bg = os.path.join(bg_dir, random.choice(bg_list))
    background = cv2.imread(bg)
    background = cv2.resize(background[:, 212:-212], (288, 208))

    # 影像1
    min_p1 = int(pow(0.02469 / min_p1, 0.5) * 100)
    max_p *= 100
    resize_scale1 = random.randint(min_p1, max_p) / 100
    new_width1 = int(image_width1 * resize_scale1)
    new_height1 = int(image_height1 * resize_scale1)
    resized_image1 = cv2.resize(image1, (new_width1, new_height1))
    max_x_offset1 = 154 - new_width1
    max_y_offset1 = 114 - new_height1
    x_offset1 = random.randint(0, max_x_offset1)
    y_offset1 = random.randint(0, max_y_offset1)
    # 影像2
    min_p2 = int(pow(0.02469 / min_p2, 0.5) * 100)
    resize_scale2 = random.randint(min_p2, max_p) / 100
    new_width2 = int(image_width2 * resize_scale2)
    new_height2 = int(image_height2 * resize_scale2)
    resized_image2 = cv2.resize(image2, (new_width2, new_height2))
    max_x_offset2 = 154 - new_width2
    max_y_offset2 = 208 - new_height2
    x_offset2 = random.randint(0, max_x_offset2)
    y_offset2 = random.randint(94, max_y_offset2)
    # 影像3
    min_p3 = int(pow(0.02469 / min_p3, 0.5) * 100)
    resize_scale3 = random.randint(min_p3, max_p) / 100
    # print(resize_scale3)
    new_width3 = int(image_width3 * resize_scale3)
    new_height3 = int(image_height3 * resize_scale3)
    resized_image3 = cv2.resize(image3, (new_width3, new_height3))
    max_x_offset3 = 288 - new_width3
    max_y_offset3 = 114 - new_height3
    x_offset3 = random.randint(134, max_x_offset3)
    y_offset3 = random.randint(0, max_y_offset3)
    # 影像4
    min_p4 = int(pow(0.02469 / min_p4, 0.5) * 100)
    resize_scale4 = random.randint(min_p4, max_p) / 100
    new_width4 = int(image_width4 * resize_scale4)
    new_height4 = int(image_height4 * resize_scale4)
    resized_image4 = cv2.resize(image4, (new_width4, new_height4))
    max_x_offset4 = 288 - new_width4
    max_y_offset4 = 208 - new_height4
    x_offset4 = random.randint(134, max_x_offset4)
    y_offset4 = random.randint(94, max_y_offset4)

    
    if(random.random()>0.9):
        resized_image1 = add_random_lens_flare(resized_image1)
    if(random.random()>0.9):
        resized_image1 = simulate_backlighting(resized_image1)
    if(random.random()>0.9):
        resized_image2 = add_random_lens_flare(resized_image2)
    if(random.random()>0.9):
        resized_image2 = simulate_backlighting(resized_image2)
    if(random.random()>0.9):
        resized_image3 = add_random_lens_flare(resized_image3)
    if(random.random()>0.9):
        resized_image3 = simulate_backlighting(resized_image3)
    if(random.random()>0.9):
        resized_image4 = add_random_lens_flare(resized_image4)
    if(random.random()>0.9):
        resized_image4 = simulate_backlighting(resized_image4)

    # 將縮放後的影像貼在背景上
    background[y_offset1:y_offset1 + new_height1, x_offset1:x_offset1 + new_width1] = resized_image1
    background[y_offset2:y_offset2 + new_height2, x_offset2:x_offset2 + new_width2] = resized_image2
    background[y_offset3:y_offset3 + new_height3, x_offset3:x_offset3 + new_width3] = resized_image3
    background[y_offset4:y_offset4 + new_height4, x_offset4:x_offset4 + new_width4] = resized_image4
    new_bbox1 = modify_paste_json(json_path1, resize_scale1, x_offset1, y_offset1)
    new_bbox2 = modify_paste_json(json_path2, resize_scale2, x_offset2, y_offset2)
    new_bbox3 = modify_paste_json(json_path3, resize_scale3, x_offset3, y_offset3)
    new_bbox4 = modify_paste_json(json_path4, resize_scale4, x_offset4, y_offset4)

    # 顯示結果影像和新的 bounding box
    # cv2.rectangle(background, (new_bbox1[0], new_bbox1[1]), (new_bbox1[0] + new_bbox1[2], new_bbox1[1] + new_bbox1[3]), (0, 255, 0), 2)
    # cv2.rectangle(background, (new_bbox2[0], new_bbox2[1]), (new_bbox2[0] + new_bbox2[2], new_bbox2[1] + new_bbox2[3]), (0, 255, 0), 2)
    # cv2.rectangle(background, (new_bbox3[0], new_bbox3[1]), (new_bbox3[0] + new_bbox3[2], new_bbox3[1] + new_bbox3[3]), (0, 255, 0), 2)
    # cv2.rectangle(background, (new_bbox4[0], new_bbox4[1]), (new_bbox4[0] + new_bbox4[2], new_bbox4[1] + new_bbox4[3]), (0, 255, 0), 2)

    # cv2.imshow('Result', background)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # 保存結果影像
    output_json = os.path.join(output_path, 'json', img1.split('.')[0] + '_' + img2.split('.')[0] + '_' + img3.split('.')[0] + '_' + img4.split('.')[0] + '.json')
    # print("output_json: ", output_json)
    output_img = os.path.join(output_path, img1.split('.')[0] + '_' + img2.split('.')[0] + '_' + img3.split('.')[0] + '_'+ img4)
    # print("output_img: ", output_img)
    bbox = [new_bbox1, new_bbox2, new_bbox3, new_bbox4]
    data = {"bboxes": bbox}
    with open(output_json, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    # 保存結果影像
    cv2.imwrite(output_img, background)
